<a href="https://colab.research.google.com/github/khietvuarong/Coding-Exercise---Prompt-Engineering/blob/main/MinhKhietLe_Self_Reflection_Prompt_for_Improving_Output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# ==============================
# 🔎 SAFE MODEL API CHECK
# ==============================

!pip install -q google-generativeai

import google.generativeai as genai
from google.colab import userdata

API_KEY = userdata.get('Ex3SecretKey')

if API_KEY is None:
    raise ValueError("API key not found. Set Ex3SecretKey in Colab Secrets.")

genai.configure(api_key=API_KEY)

print("Fetching available Gemini models...\n")

valid_models = []

for m in genai.list_models():
    model_name = m.name.split("/")[-1]

    # Only keep models that:
    # 1. Support generateContent
    # 2. Are NOT audio models
    # 3. Are NOT embedding models
    if (
        "generateContent" in m.supported_generation_methods
        and "audio" not in model_name.lower()
        and "embedding" not in model_name.lower()
    ):
        valid_models.append(model_name)

if not valid_models:
    raise ValueError("No valid text generation Gemini models found.")

print("✅ Valid Text Generation Models:\n")
for vm in valid_models:
    print(" -", vm)

# Pick safest default
SELECTED_MODEL = valid_models[0]

print(f"\n✅ Auto-selected model: {SELECTED_MODEL}")

Fetching available Gemini models...

✅ Valid Text Generation Models:

 - gemini-2.5-flash
 - gemini-2.5-pro
 - gemini-2.0-flash
 - gemini-2.0-flash-001
 - gemini-2.0-flash-exp-image-generation
 - gemini-2.0-flash-lite-001
 - gemini-2.0-flash-lite
 - gemini-2.5-flash-preview-tts
 - gemini-2.5-pro-preview-tts
 - gemma-3-1b-it
 - gemma-3-4b-it
 - gemma-3-12b-it
 - gemma-3-27b-it
 - gemma-3n-e4b-it
 - gemma-3n-e2b-it
 - gemini-flash-latest
 - gemini-flash-lite-latest
 - gemini-pro-latest
 - gemini-2.5-flash-lite
 - gemini-2.5-flash-image
 - gemini-2.5-flash-lite-preview-09-2025
 - gemini-3-pro-preview
 - gemini-3-flash-preview
 - gemini-3.1-pro-preview
 - gemini-3.1-pro-preview-customtools
 - gemini-3-pro-image-preview
 - nano-banana-pro-preview
 - gemini-robotics-er-1.5-preview
 - gemini-2.5-computer-use-preview-10-2025
 - deep-research-pro-preview-12-2025

✅ Auto-selected model: gemini-2.5-flash


In [21]:
# ==============================
# 🤖 AI SETUP
# ==============================

USE_FALLBACK = False

def generate_response(prompt):
    try:
        model = genai.GenerativeModel(SELECTED_MODEL)
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        if USE_FALLBACK:
            print("Primary failed. Trying gemini-1.5-flash fallback...")
            try:
                fallback_model = genai.GenerativeModel("gemini-1.5-flash")
                response = fallback_model.generate_content(prompt)
                return response.text
            except:
                raise RuntimeError("Fallback model also failed.")
        else:
            raise e

print(f"✅ Model ready: {SELECTED_MODEL}")

✅ Model ready: gemini-2.5-flash


In [26]:
# ==============================
# 📝 EXERCISE 3
# ==============================

from google.colab import files
import threading
import time
import sys

# ==============================
# 🤖 Animated Loading Spinner
# ==============================

def animated_loading(message="🤖 Analyzing"):
    stop_event = threading.Event()

    def run():
        spinner = ["|", "/", "-", "\\"]
        i = 0
        while not stop_event.is_set():
            sys.stdout.write(f"\r{message} {spinner[i % len(spinner)]}")
            sys.stdout.flush()
            i += 1
            time.sleep(0.2)

    thread = threading.Thread(target=run)
    thread.start()
    return stop_event

def stop_loading(loader):
    loader.set()
    time.sleep(0.3)
    print("\r" + " " * 60 + "\r", end="")

# ==============================
# INPUT OPTIONS
# ==============================

print("Choose input option:")
print("1 - Use default AI text")
print("2 - Write your own text")
print("3 - Upload a .txt file")

choice = input("Enter 1, 2, or 3: ")

if choice == "1":
    original_text = """
Artificial Intelligence (AI) is transforming industries by automating tasks,
improving decision-making, and enabling new products and services.
Companies use AI for customer support chatbots, predictive analytics,
fraud detection, and personalized marketing. While AI increases efficiency,
it also raises concerns about job displacement and ethical risks.
"""
    print("\nUsing default text.\n")

elif choice == "2":
    print("\nPaste your text below:")
    original_text = input()

elif choice == "3":
    print("\nUpload a .txt file:")
    uploaded = files.upload()
    file_name = list(uploaded.keys())[0]
    original_text = uploaded[file_name].decode("utf-8")
    print(f"\nLoaded file: {file_name}\n")

else:
    raise ValueError("Invalid choice. Please run again.")

# ==============================
# STEP 1 — INITIAL SUMMARY
# ==============================

initial_prompt = f"""
You are a professional business writer.

Summarize the text below.

Requirements:
- Maximum 80 words
- Business audience
- Single paragraph
- No bullet points

TEXT:
{original_text}
"""

loader = animated_loading("🤖 Generating summary")
initial_summary = generate_response(initial_prompt)
stop_loading(loader)

print("\n===== ORIGINAL SUMMARY =====\n")
print(initial_summary)

# ==============================
# STEP 2 — SELF REFLECTION
# ==============================

reflection_prompt = f"""
You are reviewing your own summary.

Evaluate using ALL criteria:

1. Accuracy – Does it reflect the original text?
2. Clarity – Is it easy to understand?
3. Audience – Appropriate for business readers?
4. Length – Under 80 words?
5. Format – Single paragraph, no bullet points?
6. Completeness – Any missing important ideas?

Instructions:
- Provide bullet-point critique first.
- Then provide improved summary under 80 words.
- Revised summary must follow all requirements.

Output format:

CRITIQUE:
- ...

REVISED SUMMARY:
<improved summary>

ORIGINAL TEXT:
{original_text}

SUMMARY TO REVIEW:
{initial_summary}
"""

loader = animated_loading("🤖 Reviewing and improving")
improved_output = generate_response(reflection_prompt)
stop_loading(loader)

print("\n===== CRITIQUE + IMPROVED SUMMARY =====\n")
print(improved_output)

print("\n✅ Exercise 3 completed successfully.")

Choose input option:
1 - Use default AI text
2 - Write your own text
3 - Upload a .txt file
Enter 1, 2, or 3: 2

Paste your text below:
"Copyright Disclaimer, Under Section 107 of the Copyright Act 1976, allowance is made for 'fair use' for purposes such as criticism, comment, news reporting, teaching, scholarship, and research. Fair use is a use permitted by copyright statute that might otherwise be infringing. Non-profit, educational or personal use tips the balance in favor of fair use." Mọi vấn đề vi phạm bản quyền xin liên hệ trực tiếp  (for copyright issue please contect with)  
                                                            
===== ORIGINAL SUMMARY =====

Section 107 of the Copyright Act 1976 establishes 'fair use,' allowing certain uses of copyrighted material that would otherwise be infringing. This includes purposes like criticism, comment, news reporting, teaching, scholarship, and research. Non-profit, educational, or personal use generally tips the balance towa